<h1>Classical Methods - Results</h1>

<h5>Importing Packages</h5>

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
try:
    from google.colab import drive
    drive.mount('/drive', force_remount=False)
    work_folder = '/drive/My Drive/Colab Notebooks/Experiments/Classical Machine Learning/'
except:
    work_folder = os.getcwd()

<h5>Structuring Data</h5>

In [ ]:
methods = ["KMC", "KNN", "SVM"]
fields = { methods[0]: "clusters", methods[1]: "neighbors", methods[2]: "kernel" }

execution_logs = {}

for method in methods:

    execution_logs[method] = []
    
    for experiment in range(1, 4):
        file = os.path.join(work_folder, method, "Experiment " + str(experiment), "experiment-execution-log.csv")
        content = pd.read_csv(file)
        execution_logs[method].append(content)

In [ ]:
data = {}

for method in methods:
    
    data[method] = []

    for experiment in range(0, 3):

        series = []
        field = fields[method]
        variations = len(execution_logs[method][experiment][field].unique())
        
        for index in range(0, 5): # Window numbers

            start = index * variations
            end = (index + 1) * variations
            values = execution_logs[method][experiment].iloc[start:end, :]

            row = pd.Series()

            for i in range(0, variations):
                
                line = values.iloc[i, :]

                row["window"] = line["window"]
                
                column = "Experiment "
                column += str(int(line["experiment"]))
                column += " - " 
                column += field
                column += " " 

                if isinstance(line[field], float):
                    column += str(int(line[field]))
                else:
                    column += str(line[field])
                
                row[column] = line["val_acc"]
                row.index.name = str(line["window"])

            series.append(row)
        
        data[method].append(pd.DataFrame(series))

In [ ]:
sumarized = {}
datacopy = {}

for method in methods:

    datacopy[method] = []

    for experiment in range(0, 3):
        datacopy[method].append(data[method][experiment].copy())
        datacopy[method][experiment].columns = [x[x.find("-") + 1:].strip() for x in datacopy[method][experiment].columns]

    sumarized[method] = datacopy[method][0].append(datacopy[method][1], ignore_index=True).append(datacopy[method][2], ignore_index=True).groupby('window', as_index=False).mean()

<h5>Sumarized Results</h5>

<h6>KMC</h6>

In [ ]:
pd.set_option("float_format", '{:0.2f}%'.format)

In [ ]:
data = sumarized['KMC'].copy()
data = data.transpose()
data["average"] = data.mean(axis=1)
data * 100

In [ ]:
pd.set_option("float_format", '{:0.4f}'.format)

In [ ]:
columns = [x for x in sumarized['KMC'].columns if x != "window"]
columnMax = sumarized['KMC'][columns].idxmax(axis=1)[0]
sumarized['KMC'].iloc[sumarized['KMC'][[columnMax]].idxmax()][["window", columnMax]]

In [ ]:
[x[x["window"] == 300] for x in datacopy["KMC"]]

<h6>KNN</h6>

In [ ]:
pd.set_option("float_format", '{:0.2f}%'.format)

In [ ]:
data = sumarized['KNN'].copy()
data["average"] = data[[x for x in data.columns if x != "window"]].mean(axis=1)
data = data.transpose()
data["average"] = data.mean(axis=1)
data * 100

In [ ]:
pd.set_option("float_format", '{:0.4f}'.format)

In [ ]:
columns = [x for x in sumarized['KNN'].columns if x != "window"]
columnMax = sumarized['KNN'][columns].idxmax(axis=1)[0]
sumarized['KNN'].iloc[sumarized['KNN'][[columnMax]].idxmax()][["window", columnMax]]

In [ ]:
[x[["window", "neighbors 5"]][x["window"] == 200] for x in datacopy["KNN"]]

<h6>SVM</h6>

In [ ]:
pd.set_option("float_format", '{:0.2f}%'.format)

In [ ]:
data = sumarized['SVM'].copy()
data["average"] = data[[x for x in data.columns if x != "window"]].mean(axis=1)
data = data.transpose()
data["average"] = data.mean(axis=1)
data * 100

In [ ]:
pd.set_option("float_format", '{:0.4f}'.format)

In [ ]:
columns = [x for x in sumarized['SVM'].columns if x != "window"]
columnMax = sumarized['SVM'][columns].idxmax(axis=1)[0]
sumarized['SVM'].iloc[sumarized['SVM'][[columnMax]].idxmax()][["window", columnMax]]

In [ ]:
[x[["window", "kernel rbf"]][x["window"] == 200] for x in datacopy["SVM"]]

<h5>Plots</h5>

In [ ]:
markers = ["o", "s", "v", "p", "P", "D", "X", "<", ">"]

In [ ]:
def plotOne(data, figsize = (6, 3), bbox_to_anchor=(0.5, -0.40)):

    num_plots = 1
    
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(num_plots, 1, hspace=0.25)
    ax = fig.add_subplot(gs[0, 0])
    ax.set_ylabel('Accuracy (%)')
    ax.set_xlabel('Window Size')
    ax.grid()

    fig.subplots_adjust(bottom=0.2)

    legends = []

    for experiment in range(0, 3):

        columns = [x for x in data[experiment].columns if x != "window"]
        x = data[experiment]["window"]

        for col in columns:
            legends.append(col)

        for i in range(0, len(columns)):

            y = data[experiment][columns].iloc[:, i] * 100
            ax.plot(x, y, '-', marker=markers[experiment], markersize=8, linewidth=1)

    plt.legend(legends, loc='lower center', bbox_to_anchor=bbox_to_anchor, ncol=3)

In [ ]:
def plotMany(data, figsize = (6,9), bbox_to_anchor=(0.5, -0.45)):

    num_plots = 3
    
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(num_plots, 1, hspace=0.25)

    axes = []
    legends = []

    for i in range(0, num_plots):

        ax2 = fig.add_subplot(gs[i, 0])
        ax2.yaxis.tick_right()
        ax2.set_yticks([])
        ax2.yaxis.set_label_position("right")
        ax2.set_ylabel("Experiment " + str(i+1))

        ax = fig.add_subplot(gs[i, 0], sharex=ax2, frameon=False)
        axes.append(ax)
        ax.grid()
        ax.set_ylabel('Accuracy (%)')

        if i == num_plots - 1:
            ax.set_xlabel('Window Size')
        
    for experiment in range(0, 3):

        columns = [x for x in data[experiment].columns if x != "window"]
        x = data[experiment]["window"]

        for col in columns:
            legends.append(col)

        for i in range(0, len(columns)):

            y = data[experiment][columns].iloc[:, i] * 100
            axes[experiment].plot(x, y, '-', marker=markers[i], markersize=8, linewidth=1)

    plt.legend([x[x.find("-") + 1:].strip() for x in legends], loc='lower center', bbox_to_anchor=bbox_to_anchor, ncol=3)

In [ ]:
plotOne(data["KMC"], (10, 4), (0.5, -0.4))
plt.savefig('kmc_results.png')

In [ ]:
plotMany(data["KNN"], (10, 12), (0.5, -0.5))
plt.savefig('knn_results.png')

In [ ]:
plotMany(data["SVM"], (10, 12), (0.5, -0.5))
plt.savefig('svm_results.png')